In [1]:
from pathlib import Path
import requests
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
file_path = Path('./shakespeare.txt')
if not file_path.exists():
    print('Downloading...')
    with requests.get(url, stream=True) as r:
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)
    print(f'Saving to {file_path}')
else:
    print("Dataset exists")


Dataset exists


In [3]:
with open('shakespeare.txt') as f:
    text = f.read() 
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
itos = {i:c for i, c in enumerate(chars)}
ctoi = {c:i for i, c in itos.items()}
encode = lambda s: [ctoi[c] for c in s]
decode = lambda a: ''.join([itos[i] for i in a])

t = 'Hello There!'
print(encode(t))
print(decode(encode(t)))

[20, 43, 50, 50, 53, 1, 32, 46, 43, 56, 43, 2]
Hello There!


In [6]:
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
data[:100]

torch.Size([1115394]) torch.int64


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [7]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [8]:
torch.manual_seed(1337)
batch_size = 4
context_length = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    return x, y

xb, yb = get_batch(train_data)
xb, yb = get_batch('train')
# print('inputs:')
# print(xb.shape)
# print(xb)
# print('targets:')
# print(yb.shape)
# print(yb)
# print('----')
# for b in range(batch_size): # batch dimension
#     for t in range(context_length): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is: {context.tolist()}; the target is: {target}")

In [9]:
torch.manual_seed(1337)

class BigramLM(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs, targets=None):
        loss = None
        logits = self.embed(inputs)
        
        if targets is None:
            return logits, loss 
        
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, input, max_num_tokens=50):
        for _ in range(max_num_tokens):
            logits, _ = self(input)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            preds = torch.multinomial(probs, num_samples=1)
            input = torch.concatenate((input, preds), dim=1)
        return input

In [10]:
model = BigramLM(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

input = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(input)[0].tolist()))

torch.Size([32, 65])
tensor(4.4150, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHj


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
batch_size = 128
def train(model, max_steps=int(1e3)):
    losses = []
    model.train()
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    for step in range(max_steps):
        xb, yb = get_batch('train')
        xb = xb.to(device)
        yb = yb.to(device)
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        losses.append(loss)
    model.eval()
    print(torch.tensor(losses).mean())

In [13]:
# train(model, max_steps=int(1e5))
# torch.save(model.state_dict(), './bigram.pt')
# del model

2.5075032711029053


In [14]:
model = BigramLM(vocab_size)
model.load_state_dict(torch.load('./bigram.pt'))
model.eval()
model.to(device)

BigramLM(
  (embed): Embedding(65, 65)
)

In [15]:
input = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(input.to(device), max_num_tokens=1000)[0].tolist()))




CExthy brid owindakis s, bth

Hiset bube d e.
S:
O:
IS:
Falatanss:
Wanthar u qur, vet?
F dilasoate awice my.

Hastarom oroup
Yowhthetof isth ble mil ndill, ath iree sengmin lat Heriliovets, and Win nghir.
Thanousel lind me l.
HAshe ce hiry:
Supr aisspllw y.
Hentoul n Boopetelaves
MP:

Pl, d mothakleo Windo whth eisbyo the m dourive we higend t so mower; te

AN ad nterupt f s ar igr t m:

Thiny aleronth,
Mad
Whed my o myr f-bube!
KENoby ak
Sadsal thes ghesthidin cour ay aney Iry ts I fr t ce.
Jonge ton, bemary.
Yof 'sour mend sora anghy t--pond betwe ten.
Wand thot sulin s th llety ome.
I muco ffepyotssthecas l.
TAn.
Mourethal wave.
se ed Pe bene ovetour?
Cassce oros cok hedin tie s ind aus te fe f tas ny, ct Clo gscest hes,
A: du he n, soxcone.

NENobeakes aghercobun ws m k s withoumas Fond t s wllo INour id, mersed
Fourd?
TI idurd po venond, d Cad ty
K:
BIUSoou tiund thornofen e sutan wiporthare whanot, sitthers, spe Bllellke, on s h O, t pan, ce wat d tive wout ir f; u;

Fe cen ou

In [16]:
import torch
B, T, C = 4, 8, 2
x = torch.round(torch.randn(B, T, C), decimals=3)
x.shape

torch.Size([4, 8, 2])

In [17]:
x[0]

tensor([[ 0.4330, -1.4870],
        [-0.4600,  0.2510],
        [-3.2010,  1.0920],
        [ 1.2770,  0.2270],
        [ 0.2660,  0.4500],
        [ 0.7730,  0.8520],
        [ 0.5340,  1.3030],
        [-0.0610,  0.7260]])

In [18]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b, t] = xprev.mean(dim=0)

In [19]:
xbow[0]

tensor([[ 0.4330, -1.4870],
        [-0.0135, -0.6180],
        [-1.0760, -0.0480],
        [-0.4877,  0.0207],
        [-0.3370,  0.1066],
        [-0.1520,  0.2308],
        [-0.0540,  0.3840],
        [-0.0549,  0.4268]])

In [20]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x
xbow2[0]

tensor([[ 0.4330, -1.4870],
        [-0.0135, -0.6180],
        [-1.0760, -0.0480],
        [-0.4877,  0.0208],
        [-0.3370,  0.1066],
        [-0.1520,  0.2308],
        [-0.0540,  0.3840],
        [-0.0549,  0.4268]])

In [21]:
torch.allclose(xbow, xbow2)

True

In [22]:
from torch.nn import functional as F
tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
xbow3[0]

tensor([[ 0.4330, -1.4870],
        [-0.0135, -0.6180],
        [-1.0760, -0.0480],
        [-0.4877,  0.0208],
        [-0.3370,  0.1066],
        [-0.1520,  0.2308],
        [-0.0540,  0.3840],
        [-0.0549,  0.4268]])

In [18]:
import torch
import torch.nn as nn

torch.manual_seed(43)

B, T, C = 4, 8, 32
x = torch.round(torch.randn(B, T, C), decimals=3) # (B, T, C)

# Single Self Attention
H = 16 # Number of heads
query = nn.Linear(C, H, bias=False)
key = nn.Linear(C, H, bias=False)
value = nn.Linear(C, H, bias=False)

q = query(x) # B, T, C @ C, H --> B, T, H
k = key(x)   # B, T, C @ C, H --> B, T, H
v = value(x) # B, T, H
wei = q @ k.transpose(-2, -1) # B, T, T
print(wei.shape)
# print(wei)

from torch.nn import functional as F
tril = torch.tril(torch.ones((T, T)))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1) # (T, T)
xbow = wei @ v # --> B, T, T @ T, H  --> B, T, H
print(xbow.shape)

torch.Size([4, 8, 8])
torch.Size([4, 8, 16])


In [20]:
xbow

tensor([[[ 2.4819e-02, -9.9487e-02,  1.0058e-01, -6.7135e-02,  6.4086e-02,
           1.2452e-03, -7.3203e-02, -2.6080e-02, -4.8787e-02, -1.2246e-02,
           4.4570e-03, -4.8490e-02,  3.3838e-02,  1.0091e-01, -6.3384e-03,
           1.4211e-03],
         [-1.0222e-02, -3.7330e-02,  8.7879e-03,  1.9920e-01, -1.1404e-01,
          -8.4670e-02, -1.1518e-01, -9.5564e-02, -1.1023e-01,  2.4883e-01,
          -2.7324e-01, -1.0239e-02,  4.9485e-02, -1.2546e-01,  6.5045e-02,
           8.1755e-02],
         [ 1.5698e-01, -2.4639e-01,  3.7068e-01,  3.7799e-03,  2.8637e-02,
           2.9191e-02, -1.5189e-01, -5.7714e-02, -7.7026e-03,  1.9733e-01,
          -3.5101e-02,  3.2930e-02,  1.2442e-01,  1.4528e-01, -1.4823e-01,
          -2.7407e-01],
         [ 1.3579e-02, -1.3931e-01,  1.8807e-01,  1.7672e-01, -1.5920e-01,
          -2.4711e-02, -1.3165e-01, -6.0952e-02, -5.7402e-02,  2.9385e-01,
          -1.4252e-01,  7.8842e-02,  1.0188e-01, -2.4537e-02, -2.1866e-02,
          -3.3277e-02],
    